In [1]:
import os
from PIL import Image
from collections import defaultdict
from tqdm import tqdm

# --- 설정 ---
# ▼▼▼ 파일 개수를 셀 폴더의 경로를 입력하세요 ▼▼▼
ROOT_DIRECTORY = '../데이터/data/kfood'


# --- 메인 실행 로직 ---
print(f"'{os.path.abspath(ROOT_DIRECTORY)}' 경로의 실제 파일 형식 분석을 시작합니다...\n")

# 결과를 저장할 딕셔너리: {폴더 경로: {'jpegs': 개수, 'others': 개수}}
folder_stats = defaultdict(lambda: defaultdict(int))
total_jpeg_count = 0
total_other_count = 0

# '그 외 파일'을 담을 리스트
files_to_delete = []

# os.walk를 사용하여 모든 폴더와 파일을 순회
# 먼저 전체 파일 목록을 만들어 tqdm에 사용
all_files = [os.path.join(r, f) for r, _, fs in os.walk(ROOT_DIRECTORY) for f in fs if os.path.isfile(os.path.join(r, f))]

for file_path in tqdm(all_files, desc="파일 형식 분석 중"):
    dirpath = os.path.dirname(file_path)
    is_jpeg = False

    try:
        with Image.open(file_path) as img:
            if img.format == 'JPEG':
                is_jpeg = True
    except (IOError, SyntaxError):
        is_jpeg = False

    if is_jpeg:
        folder_stats[dirpath]['jpegs'] += 1
        total_jpeg_count += 1
    else:
        # '그 외 파일' 리스트에 추가
        files_to_delete.append(file_path)
        folder_stats[dirpath]['others'] += 1
        total_other_count += 1

# --- 파일 삭제 로직 추가 ---
print(f"\n🗑️ 그 외 파일 {len(files_to_delete)}개를 삭제합니다.")
for file_path in tqdm(files_to_delete, desc="파일 삭제 중"):
    try:
        os.remove(file_path)
    except OSError as e:
        print(f"오류: {file_path} 삭제 실패 - {e.strerror}")

# --- 결과 출력 ---
print(f"\n📂 총 파일 개수: {total_jpeg_count + total_other_count}개")
print(f"  - 실제 JPEG 파일: {total_jpeg_count}개")
print(f"  - 그 외 파일: {total_other_count}개")
print("-" * 40)

if not folder_stats:
    print("분석할 파일이 없습니다.")
else:
    # 폴더 경로 순으로 정렬하여 출력
    for folder, counts in sorted(folder_stats.items()):
        folder_total = counts.get('jpegs', 0) + counts.get('others', 0)
        print(f"📁 폴더: {folder} (총 {folder_total}개)")

        if 'jpegs' in counts:
            print(f"  - 실제 JPEG 파일: {counts['jpegs']}개")
        if 'others' in counts:
            print(f"  - 그 외 파일: {counts['others']}개")
        print()

'/Users/ihanjo/Documents/미니 프로젝트/한국음식 예측/데이터/data/kfood' 경로의 실제 파일 형식 분석을 시작합니다...



파일 형식 분석 중: 100%|██████████| 150610/150610 [00:19<00:00, 7554.55it/s]



🗑️ 그 외 파일 0개를 삭제합니다.


파일 삭제 중: 0it [00:00, ?it/s]


📂 총 파일 개수: 150610개
  - 실제 JPEG 파일: 150610개
  - 그 외 파일: 0개
----------------------------------------
📁 폴더: ../데이터/data/kfood/가지볶음 (총 1000개)
  - 실제 JPEG 파일: 1000개

📁 폴더: ../데이터/data/kfood/간장게장 (총 1001개)
  - 실제 JPEG 파일: 1001개

📁 폴더: ../데이터/data/kfood/갈비구이 (총 1000개)
  - 실제 JPEG 파일: 1000개

📁 폴더: ../데이터/data/kfood/갈비찜 (총 1000개)
  - 실제 JPEG 파일: 1000개

📁 폴더: ../데이터/data/kfood/갈비탕 (총 1000개)
  - 실제 JPEG 파일: 1000개

📁 폴더: ../데이터/data/kfood/갈치구이 (총 1000개)
  - 실제 JPEG 파일: 1000개

📁 폴더: ../데이터/data/kfood/갈치조림 (총 1005개)
  - 실제 JPEG 파일: 1005개

📁 폴더: ../데이터/data/kfood/감자전 (총 1001개)
  - 실제 JPEG 파일: 1001개

📁 폴더: ../데이터/data/kfood/감자조림 (총 1001개)
  - 실제 JPEG 파일: 1001개

📁 폴더: ../데이터/data/kfood/감자채볶음 (총 1000개)
  - 실제 JPEG 파일: 1000개

📁 폴더: ../데이터/data/kfood/감자탕 (총 1003개)
  - 실제 JPEG 파일: 1003개

📁 폴더: ../데이터/data/kfood/갓김치 (총 1000개)
  - 실제 JPEG 파일: 1000개

📁 폴더: ../데이터/data/kfood/건새우볶음 (총 1000개)
  - 실제 JPEG 파일: 1000개

📁 폴더: ../데이터/data/kfo

In [ ]:
import os
import tensorflow as tf
from tqdm import tqdm

# --- 설정 ---
# ▼▼▼ 검색을 시작할 폴더의 경로를 입력하세요 ▼▼▼
ROOT_DIRECTORY = '../데이터/data/kfood'


# --- 메인 실행 로직 ---
print(f"'{os.path.abspath(ROOT_DIRECTORY)}' 경로에서 TensorFlow를 이용해 4채널 이미지를 검색합니다...\n")

# 검색할 이미지 확장자 목록 (소문자로)
target_extensions = ('.png', '.gif', '.tiff', '.webp', '.bmp')

# 4채널 이미지 파일 경로를 저장할 리스트
four_channel_files = []

# os.walk를 사용하여 모든 폴더와 파일을 순회
total_files = sum(len(files) for _, _, files in os.walk(ROOT_DIRECTORY))

with tqdm(total=total_files, desc="이미지 채널 확인 중") as pbar:
    for dirpath, _, filenames in os.walk(ROOT_DIRECTORY):
        for filename in filenames:
            pbar.update(1) # 진행률 1 증가
            
            # 파일 확장자가 대상 목록에 있는지 확인 (대소문자 무시)
            if filename.lower().endswith(target_extensions):
                file_path = os.path.join(dirpath, filename)
                try:
                    # TensorFlow로 파일을 바이트 단위로 읽기
                    img_bytes = tf.io.read_file(file_path)
                    
                    # TensorFlow로 이미지 디코딩
                    # channels=0 옵션은 이미지 파일의 기본 채널 수를 따르도록 함
                    image_tensor = tf.io.decode_image(img_bytes, channels=0, expand_animations=False)
                    
                    # 텐서의 마지막 차원(채널 수)을 확인
                    if image_tensor.shape[-1] == 4:
                        four_channel_files.append(file_path)
                        
                except Exception:
                    # TensorFlow가 디코딩할 수 없는 손상된 파일 등은 건너뜁니다.
                    continue

# --- 결과 출력 ---
print("\n" + "-" * 50)
if not four_channel_files:
    print("✅ 4채널 이미지를 찾을 수 없었습니다.")
else:
    print(f"⚠️ 총 {len(four_channel_files)}개의 4채널(RGBA) 이미지를 찾았습니다.")
    print("--- 파일 목록 ---")
    for f_path in four_channel_files:
        print(f_path)
print("=" * 50)